In [1]:
import pandas as pd

In [2]:
import os
os.chdir("../../")
df = pd.read_csv("data/dataset/after_preprocessing.csv")
df.head()

,Unnamed: 0,Категория,Краткое описание,Описание,Решение,Аналитика 1,Аналитика 2,Аналитика 3
0,1,7,Без СППР. FW: 2.0 _АО_ нет графы количества,В авансовых отчетах программы XС X.X нет графы...,добрый день! Доработка функционала XC ERP X.X...,"Казначейство, взаиморасчеты",Расчеты с персоналом,Авансовые отчеты
1,2,7,Без СППР. 2023 ЗИ - 1.2 ФУНКЦИОНАЛ 1С ЕРП2. ...,Система «срезает» часть номера ЗРС при отражен...,Доработка функционала возможна в рамках запрос...,"Казначейство, взаиморасчеты",Казначейство,Заявки на оплату
2,3,7,Без СППР. Настройки для отчета по дебиторско...,Добрый день! Отправляю обращение проектной ко...,Добрый день. а. В системе созданы отчеты по де...,"Казначейство, взаиморасчеты",Взаиморасчеты,Отчеты
3,4,7,Без СППР. 1.2 ФУНКЦИОНАЛ 1С ЕРП2: в дополн...,"Добрый день, Признак ``Списать на расходы`` ...",Доработка функционала возможна в рамках запрос...,"Казначейство, взаиморасчеты",Казначейство,Платежные документы
4,5,7,Без СППР. 1.2 ФУНКЦИОНАЛ 1С ЕРП2. Актуально...,XX.XX.XXXX SDXXXXXXXX требуется уточнение - во...,"Добрый день, Наталья Сергеевна Доработка функ...","Казначейство, взаиморасчеты",Казначейство,Платежные документы


In [3]:
df = df[["Описание", "Решение"]]
df.head()

,Описание,Решение
0,В авансовых отчетах программы XС X.X нет графы...,добрый день! Доработка функционала XC ERP X.X...
1,Система «срезает» часть номера ЗРС при отражен...,Доработка функционала возможна в рамках запрос...
2,Добрый день! Отправляю обращение проектной ко...,Добрый день. а. В системе созданы отчеты по де...
3,"Добрый день, Признак ``Списать на расходы`` ...",Доработка функционала возможна в рамках запрос...
4,XX.XX.XXXX SDXXXXXXXX требуется уточнение - во...,"Добрый день, Наталья Сергеевна Доработка функ..."


In [4]:
def concatenate_text(examples):
    return {
        "text":
        examples["Описание"]
        + " \n " +
        examples["Решение"]
    }

In [5]:
# !pip install datasets

In [6]:
from datasets import Dataset

In [7]:
df = df[:100]

In [8]:
dt = Dataset.from_pandas(df)
dt

Dataset({
    features: ['Описание', 'Решение'],
    num_rows: 100
})

In [9]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [10]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device

device(type='cpu')

In [11]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [12]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [13]:
dt = dt.map(concatenate_text)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [14]:
def readEmbeddingsArray(examples):
    string = examples["embeddings"]
    array = []
    string = string.replace("[", "")
    string = string.replace("]", "")
    for i in string.split("\n"):
        for j in i.split(" "):
            if j == "":
                continue
            array.append(float(j))
        
    return {"embeddings": array}

In [16]:
import os
if os.path.exists("data/embeddings.csv"):
    embeddings_dataset = Dataset.from_csv("data/embeddings.csv")
    embeddings_dataset = embeddings_dataset.map(readEmbeddingsArray)
    print(embeddings_dataset)
else:
    embeddings_dataset = dt.map(
        lambda x: {"embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]}
    )
    embeddings_dataset.to_csv("data/embeddings.csv")
    print(embeddings_dataset)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/home/almaz/anaconda3/envs/env/lib/python3.11/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['Описание', 'Решение', 'text', 'embeddings'],
    num_rows: 100
})


In [58]:
# install faiss-gpu through conda 

In [59]:
# !pip install faiss-cpu
# !apt install libomp-dev
# !python -m pip install --upgrade faiss faiss-cpu

In [17]:
import faiss
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['Описание', 'Решение', 'text', 'embeddings'],
    num_rows: 100
})

In [18]:
request = "Добрый день, такая проблема: документы не скачиваются. Что делать?"

In [19]:
request_embedding = get_embeddings([request]).cpu().detach().numpy()

In [20]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", request_embedding, k=5
)

In [21]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)


In [93]:
for _, row in samples_df.iterrows():
    print(row)
    print("=" * 50)
    print()

Описание      Добрый день, коллеги. Вы пишите, что документы...
Решение       Здравствуйте!  Вопрос насколько знаю не актуал...
text          Добрый день, коллеги. Вы пишите, что документы...
embeddings    [-0.04505389928817749, -0.349048912525177, -0....
scores                                                15.928876
Name: 4, dtype: object

Описание      Добрый день! Для создания резерва под обесцене...
Решение       Добрый день! Для анализа номенклатуры без движ...
text          Добрый день! Для создания резерва под обесцене...
embeddings    [-0.010267294943332672, -0.20020823180675507, ...
scores                                                15.786793
Name: 3, dtype: object

Описание            Добрый день.       Необходимо доработать...
Решение       Доработана связка объекта строительства и объе...
text                Добрый день.       Необходимо доработать...
embeddings    [0.0145293353125453, -0.2552527189254761, -0.3...
scores                                                14